In [12]:
import os
import cv2
import numpy as np
import imutils
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Crop Images

In [13]:
def crop_images(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Threshold the image
    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    # Find contours
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    if cnts:
        # Find the largest contour
        c = max(cnts, key=cv2.contourArea)

        # Find the extreme points
        extLeft = tuple(c[c[:, :, 0].argmin()][0])
        extRight = tuple(c[c[:, :, 0].argmax()][0])
        extTop = tuple(c[c[:, :, 1].argmin()][0])
        extBot = tuple(c[c[:, :, 1].argmax()][0])

        # Add contour and extreme points to the image
        img_cnt = cv2.drawContours(image.copy(), [c], -1, (0, 255, 255), 4)
        img_pnt = cv2.circle(img_cnt.copy(), extLeft, 5, (0, 0, 255), -1)
        img_pnt = cv2.circle(img_pnt, extRight, 5, (0, 255, 0), -1)
        img_pnt = cv2.circle(img_pnt, extTop, 5, (255, 0, 0), -1)
        img_pnt = cv2.circle(img_pnt, extBot, 5, (255, 255, 0), -1)

        # Apply crop
        ADD_PIXELS = 0
        new_image = image[
            extTop[1] - ADD_PIXELS : extBot[1] + ADD_PIXELS,
            extLeft[0] - ADD_PIXELS : extRight[0] + ADD_PIXELS,
        ].copy()

        return new_image
    else:
        return None

In [ ]:
def crop_and_save_images_with_plot(input_directory, output_directory):
    for root, dirs, files in os.walk(input_directory):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".png"):
                image_path = os.path.join(root, file)
                image = cv2.imread(image_path)

            # Crop the image
            cropped_image = crop_images(image)

            if cropped_image is not None:
                # Save the cropped image
                class_name = os.path.basename(os.path.dirname(image_path))
                class_directory = os.path.join(output_directory, class_name)
                os.makedirs(class_directory, exist_ok=True)
                new_image_path = os.path.join(class_directory, file)
                cv2.imwrite(new_image_path, cropped_image)

                # # Plot the sequential steps
                # fig, axs = plt.subplots(1, 2, figsize=(10, 5))
                # axs[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                # axs[0].set_title('Original Image')
                # axs[0].axis('off')

                # axs[1].imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
                # axs[1].set_title('Cropped Image')
                # axs[1].axis('off')

                # plt.tight_layout()
                # plt.show()

## Augmented Data With Plot

In [26]:
def augment_images_with_plot(input_directory, output_directory, num_images=5):
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1.0 / 255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode="nearest",
    )

    for root, dirs, files in os.walk(input_directory):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".png"):
                image_path = os.path.join(root, file)
                image = cv2.imread(image_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = image.reshape((1,) + image.shape)
                save_prefix = file.split(".")[0] + "_aug"

                class_name = os.path.basename(os.path.dirname(image_path))
                data_split = os.path.basename(
                    os.path.dirname(os.path.dirname(image_path))
                )

                class_directory = os.path.join(output_directory, data_split, class_name)
                os.makedirs(class_directory, exist_ok=True)

                i = 0
                for batch in datagen.flow(
                    image,
                    batch_size=1,
                    save_to_dir=class_directory,
                    save_prefix=save_prefix,
                    save_format="jpg",
                ):
                    i += 1
                    if i >= num_images:
                        break

In [19]:
def create_augmented_data_with_plot(input_directory, output_directory):
    os.makedirs(output_directory, exist_ok=True)
    for sub_directory in os.listdir(input_directory):
        if os.path.isdir(os.path.join(input_directory, sub_directory)):
            input_sub_directory = os.path.join(input_directory, sub_directory)
            output_sub_directory = os.path.join(output_directory, sub_directory)
            os.makedirs(output_sub_directory, exist_ok=True)
            augment_images_with_plot(input_sub_directory, output_sub_directory)

## Augmentaion

In [27]:
augmented_directory = "./augmented_data/"
train_directory = "./Brain Tumor/Training"
test_directory = "./Brain Tumor/Testing"
augment_images_with_plot(train_directory, augmented_directory)
augment_images_with_plot(test_directory, augmented_directory)

## Cropping Data

In [29]:
cropped_train = "./augmented_data/Training"
cropped_test = "./augmented_data/Testing"

cropped_train_directory = "./Images_Cropped/cropped_training/"
os.makedirs(cropped_train_directory, exist_ok=True)
crop_and_save_images_with_plot(cropped_train, cropped_train_directory)

cropped_test_directory = "./Images_Cropped/cropped_testing/"
os.makedirs(cropped_test_directory, exist_ok=True)
crop_and_save_images_with_plot(cropped_test, cropped_test_directory)